In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import warnings
warnings.filterwarnings('ignore')


In [35]:
dataset = pd.read_csv(r"D:\heart_disease.csv")
df = dataset.copy()
df.head()

,Age,Gender,Blood Pressure,Cholesterol Level,Exercise Habits,Smoking,Family Heart Disease,Diabetes,BMI,High Blood Pressure,...,High LDL Cholesterol,Alcohol Consumption,Stress Level,Sleep Hours,Sugar Consumption,Triglyceride Level,Fasting Blood Sugar,CRP Level,Homocysteine Level,Heart Disease Status
0,56.0,Male,153.0,155.0,High,Yes,Yes,No,24.991591,Yes,...,No,High,Medium,7.633228,Medium,342.0,NaN,12.969246,12.387250,No
1,69.0,Female,146.0,286.0,High,No,Yes,Yes,25.221799,No,...,No,Medium,High,8.744034,Medium,133.0,157.0,9.355389,19.298875,No
2,46.0,Male,126.0,216.0,Low,No,No,No,29.855447,No,...,Yes,Low,Low,4.440440,Low,393.0,92.0,12.709873,11.230926,No
3,32.0,Female,122.0,293.0,High,Yes,Yes,No,24.130477,Yes,...,Yes,Low,High,5.249405,High,293.0,94.0,12.509046,5.961958,No
4,60.0,Male,166.0,242.0,Low,Yes,Yes,Yes,20.486289,Yes,...,No,Low,High,7.030971,High,263.0,154.0,10.381259,8.153887,No


In [17]:
X = df.drop("Heart Disease Status", axis=1)
y = df["Heart Disease Status"]

In [18]:
# Define column types (this is just categorizing, not learning from data)
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

print(f"Total features: {X.shape[1]}")
print(f"Numerical columns: {num_features}")
print(f"Categorical columns: {cat_features}")

Total features: 20
Numerical columns: ['Age', 'Blood Pressure', 'Cholesterol Level', 'BMI', 'Sleep Hours', 'Triglyceride Level', 'Fasting Blood Sugar', 'CRP Level', 'Homocysteine Level']
Categorical columns: ['Gender', 'Exercise Habits', 'Smoking', 'Family Heart Disease', 'Diabetes', 'High Blood Pressure', 'Low HDL Cholesterol', 'High LDL Cholesterol', 'Alcohol Consumption', 'Stress Level', 'Sugar Consumption']


In [19]:
print(num_features)
print(cat_features)

['Age', 'Blood Pressure', 'Cholesterol Level', 'BMI', 'Sleep Hours', 'Triglyceride Level', 'Fasting Blood Sugar', 'CRP Level', 'Homocysteine Level']
['Gender', 'Exercise Habits', 'Smoking', 'Family Heart Disease', 'Diabetes', 'High Blood Pressure', 'Low HDL Cholesterol', 'High LDL Cholesterol', 'Alcohol Consumption', 'Stress Level', 'Sugar Consumption']


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

In [21]:
ordinal_cols = ["Exercise Habits", "Alcohol Consumption", "Stress Level", "Sugar Consumption"]
ordinal_cols = [col for col in ordinal_cols if col in cat_features]

In [22]:
nominal_cols = [col for col in cat_features if col not in ordinal_cols]
#show all columns
print(f"Numerical columns: {num_features}")
print(f"Ordinal columns: {ordinal_cols}")
print(f"Nominal columns: {nominal_cols}")


Numerical columns: ['Age', 'Blood Pressure', 'Cholesterol Level', 'BMI', 'Sleep Hours', 'Triglyceride Level', 'Fasting Blood Sugar', 'CRP Level', 'Homocysteine Level']
Ordinal columns: ['Exercise Habits', 'Alcohol Consumption', 'Stress Level', 'Sugar Consumption']
Nominal columns: ['Gender', 'Smoking', 'Family Heart Disease', 'Diabetes', 'High Blood Pressure', 'Low HDL Cholesterol', 'High LDL Cholesterol']


In [23]:
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "mean")),
    ("scalar", StandardScaler())
])

#ordinal pipeline
ordinal_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("ordinal", OrdinalEncoder(categories= [["Low", "Medium", "High"]]*len(ordinal_cols),
                             handle_unknown = "use_encoded_value", unknown_value=-1))
])

#nominal pipeline
nominal_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("onehot", OneHotEncoder(drop='first', handle_unknown='ignore'))
])



In [24]:
#Combine all preprocessing steps
preprocessor = ColumnTransformer([
    ('num',numerical_pipeline, num_features),
    ('ord', ordinal_pipeline, ordinal_cols),
    ('nom', nominal_pipeline, nominal_cols)
])

In [25]:
#defining Multiple models

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=6, random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(kernel='rbf', random_state=42, probability=True),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

print(f"Models to evaluate: {list(models.keys())}")

Models to evaluate: ['Random Forest', 'Gradient Boosting', 'Logistic Regression', 'SVM', 'KNN']


In [26]:
#Cross validation
cv_folds = 5
cv_strategy = StratifiedKFold(n_splits = cv_folds, shuffle = True, random_state = 42)


# Check class distribution
print(f"Original class distribution: {y_train.value_counts().to_dict()}")


Original class distribution: {'No': 6387, 'Yes': 1613}


In [27]:
# 7. Train and evaluate each model with cross-validation
print("\n" + "="*70)
print("CROSS-VALIDATION RESULTS")
print("="*70)

cv_results = {}
trained_models = {}

for model_name, model in models.items():
    print(f"\nEvaluating {model_name}...")
    
    # Create complete pipeline for this model
    pipeline = ImbPipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', model)
    ])
    
    # Perform cross-validation
    cv_scores = cross_val_score(
        pipeline, X_train, y_train, 
        cv=cv_strategy, 
        scoring='accuracy',
        n_jobs=1
    )
    
    # Store results
    cv_results[model_name] = {
        'cv_scores': cv_scores,
        'mean_cv_score': cv_scores.mean(),
        'std_cv_score': cv_scores.std()
    }
    
    # Train final model on full training set
    pipeline.fit(X_train, y_train)
    trained_models[model_name] = pipeline
    
    # Print cross-validation results
    print(f"CV Scores: {cv_scores}")
    print(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")



CROSS-VALIDATION RESULTS

Evaluating Random Forest...
CV Scores: [0.798125 0.798125 0.7975   0.79875  0.79875 ]
Mean CV Score: 0.7983 (+/- 0.0009)

Evaluating Gradient Boosting...
CV Scores: [0.79375  0.791875 0.795625 0.793125 0.798125]
Mean CV Score: 0.7945 (+/- 0.0044)

Evaluating Logistic Regression...
CV Scores: [0.5075   0.495    0.51625  0.496875 0.5     ]
Mean CV Score: 0.5031 (+/- 0.0157)

Evaluating SVM...
CV Scores: [0.629375 0.645    0.62125  0.641875 0.63875 ]
Mean CV Score: 0.6352 (+/- 0.0175)

Evaluating KNN...
CV Scores: [0.491875 0.47125  0.454375 0.465    0.4925  ]
Mean CV Score: 0.4750 (+/- 0.0301)


In [28]:
# 8. Rank models by cross-validation performance
print("\n" + "="*70)
print("MODEL RANKING (Cross-Validation)")
print("="*70)

# Sort models by mean CV score
sorted_models = sorted(cv_results.items(), key=lambda x: x[1]['mean_cv_score'], reverse=True)

for i, (model_name, results) in enumerate(sorted_models, 1):
    print(f"{i}. {model_name:<20} | CV Score: {results['mean_cv_score']:.4f} (+/- {results['std_cv_score']*2:.4f})")


MODEL RANKING (Cross-Validation)
1. Random Forest        | CV Score: 0.7983 (+/- 0.0009)
2. Gradient Boosting    | CV Score: 0.7945 (+/- 0.0044)
3. SVM                  | CV Score: 0.6352 (+/- 0.0175)
4. Logistic Regression  | CV Score: 0.5031 (+/- 0.0157)
5. KNN                  | CV Score: 0.4750 (+/- 0.0301)


In [29]:
# 9. Test all models on hold-out test set
print("\n" + "="*70)
print("FINAL TEST SET EVALUATION")
print("="*70)

test_results = {}

for model_name, pipeline in trained_models.items():
    # Make predictions on test set
    y_pred = pipeline.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    
    test_results[model_name] = {
        'test_accuracy': test_accuracy,
        'predictions': y_pred
    }
    
    print(f"\n{model_name}:")
    print(f"Test Accuracy: {test_accuracy:.4f}")



FINAL TEST SET EVALUATION

Random Forest:
Test Accuracy: 0.8065

Gradient Boosting:
Test Accuracy: 0.8060

Logistic Regression:
Test Accuracy: 0.4935

SVM:
Test Accuracy: 0.6295

KNN:
Test Accuracy: 0.4605


In [30]:

# 10. Select best model and show detailed results
best_model_name = max(test_results.items(), key=lambda x: x[1]['test_accuracy'])[0]
best_pipeline = trained_models[best_model_name]
best_predictions = test_results[best_model_name]['predictions']

print(f"\n" + "="*70)
print(f"BEST MODEL: {best_model_name}")
print("="*70)

print(f"\nCross-Validation Score: {cv_results[best_model_name]['mean_cv_score']:.4f}")
print(f"Test Set Accuracy: {test_results[best_model_name]['test_accuracy']:.4f}")

print(f"\nDetailed Classification Report:")
print(classification_report(y_test, best_predictions))

print(f"\nConfusion Matrix:")
cm = confusion_matrix(y_test, best_predictions)
print(cm)

# Calculate additional metrics for best model
accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
precision = cm[1,1] / (cm[1,1] + cm[0,1]) if (cm[1,1] + cm[0,1]) > 0 else 0
recall = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1] + cm[1,0]) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nDetailed Metrics for {best_model_name}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


BEST MODEL: Random Forest

Cross-Validation Score: 0.7983
Test Set Accuracy: 0.8065

Detailed Classification Report:
              precision    recall  f1-score   support

          No       0.81      1.00      0.89      1613
         Yes       0.00      0.00      0.00       387

    accuracy                           0.81      2000
   macro avg       0.40      0.50      0.45      2000
weighted avg       0.65      0.81      0.72      2000


Confusion Matrix:
[[1613    0]
 [ 387    0]]

Detailed Metrics for Random Forest:
Accuracy: 0.8065
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000


C:\Users\786\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\786\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\786\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [31]:
# 11. Feature importance (if available)
try:
    if hasattr(best_pipeline.named_steps['classifier'], 'feature_importances_'):
        feature_importance = best_pipeline.named_steps['classifier'].feature_importances_
        print(f"\nTop 5 feature importance values: {sorted(feature_importance, reverse=True)[:5]}")
    else:
        print(f"\n{best_model_name} does not provide feature importance")
except:
    print(f"\nCould not extract feature importance for {best_model_name}")



Top 5 feature importance values: [np.float64(0.12633533127297228), np.float64(0.10886516322361513), np.float64(0.10418301198064321), np.float64(0.09757929880408228), np.float64(0.08897057826708372)]


In [33]:
#12. Summary comparison table
print(f"\n" + "="*70)
print("COMPLETE COMPARISON TABLE")
print("="*70)
print(f"{'Model':<20} | {'CV Score':<15} | {'Test Score':<15} | {'Difference':<10}")
print("-" * 70)

for model_name in models.keys():
    cv_score = cv_results[model_name]['mean_cv_score']
    test_score = test_results[model_name]['test_accuracy']
    difference = abs(cv_score - test_score)
    
    print(f"{model_name:<20} | {cv_score:<15.4f} | {test_score:<15.4f} | {difference:<10.4f}")

print(f"\n" + "="*70)
print("PIPELINE EXECUTION SUMMARY")
print("="*70)
print("✅ Data split first - no leakage")
print("✅ Cross-validation performed correctly")
print("✅ Multiple models trained and compared")
print("✅ SMOTE applied for class balancing")
print("✅ StandardScaler applied to numerical features")
print("✅ All preprocessing in pipeline")
print("✅ Best model selected based on test performance")
print("="*70)

print(f"\nRecommended Model: {best_model_name}")
print(f"Expected Performance: {test_results[best_model_name]['test_accuracy']:.4f} accuracy")


COMPLETE COMPARISON TABLE
Model                | CV Score        | Test Score      | Difference
----------------------------------------------------------------------
Random Forest        | 0.7983          | 0.8065          | 0.0082    
Gradient Boosting    | 0.7945          | 0.8060          | 0.0115    
Logistic Regression  | 0.5031          | 0.4935          | 0.0096    
SVM                  | 0.6352          | 0.6295          | 0.0057    
KNN                  | 0.4750          | 0.4605          | 0.0145    

PIPELINE EXECUTION SUMMARY
✅ Data split first - no leakage
✅ Cross-validation performed correctly
✅ Multiple models trained and compared
✅ SMOTE applied for class balancing
✅ StandardScaler applied to numerical features
✅ All preprocessing in pipeline
✅ Best model selected based on test performance

Recommended Model: Random Forest
Expected Performance: 0.8065 accuracy
